# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Dataset
from azureml.core import Environment
from azureml.core import Experiment
from azureml.core import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split

## Dataset

### Overview
Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images.

It is a classification problem to find out whether the bank note is genuine or forged.

The above note and dataset have been collected from [UCI](https://archive.ics.uci.edu/ml/datasets/banknote+authentication).


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

#Getting dataset that is registered onto workspace
dataset = Dataset.get_by_name(ws, "data_banknote_authentication.csv")

train_data = dataset.to_pandas_dataframe()
print("Shape of data: ", str(train_data.shape))
train_data.head()

label = 'Class'

# Name of the experiment
experiment_name = 'azure_ml_capstone_project'
experiment=Experiment(ws, experiment_name)

experiment

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The following are reasons.

* experiment_timeout_minutes = 25 (This depicts total time to be taken by all iterations before it terminates.)
* max_concurrent_iterations = 6 (It represents maximum number of parallel execution.)
* enable_early_stopping = True (It helps terminate execution if score is not improving in the short term.)
* task = classification (This is type of execution to solve the problem.)
* primary_metric = accuracy (Accuracy being chosen which will help choose the best model selection.)
* training_data = train_data (This is train data i.e. bank note authentication.)
* label_column_name = label (The name of the label column.)
* n_cross_validations = 5 (Number of cross validation to be performed.)
* compute_target = aml_compute (Experiment to be run on compute.)

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 6,
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5,
    compute_target=aml_compute,
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
bank_note_automl_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(bank_note_automl_run).show()
bank_note_automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = bank_note_automl_run.get_output()
print(best_run, "\n")
print(best_run.get_metrics(), "\n")
print(best_model._final_estimator)

In [ ]:
#TODO: Save the best model
import joblib
joblib.dump(best_model, 'bank_note_automl_model.joblib')

# Register
model = best_run.register_model(
    model_name='bank_note_automl_model', 
    model_path='./outputs/'
)
model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service